In [ ]:
import os
from dotenv import load_dotenv
from pprint import pprint

from langchain_teddynote import logging
from models import Agent, get_rag_instance

from utils import save_output2json
from prompt import load_system_prompt, load_invoke_input

In [2]:
# .env 파일 로드
load_dotenv(dotenv_path=".env")

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
import json
import operator
import warnings
warnings.filterwarnings("ignore")

from langchain_core.messages import BaseMessage, FunctionMessage, HumanMessage
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from typing import Annotated, Sequence, TypedDict
from typing_extensions import TypedDict
import functools


# 각 에이전트와 도구에 대한 다른 노드를 생성할 것입니다. 이 클래스는 그래프의 각 노드 사이에서 전달되는 객체를 정의합니다.
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

class MultiAgentSupervisor(Agent):
    def __init__(
        self, 
        file_folder:str="./data/raw", 
        file_number:int=1, 
        chunk_size: int=500, 
        chunk_overlap: int=100, 
        search_k: int=10,   
        sample_name_searcher_model_name:str="gpt-4o",
        supervisor_model_name:str="gpt-4o",
        researcher_model_name:str="gpt-4o",
        verifier_model_name:str="gpt-4o",
        save_graph_png:bool=False,
    ):
        super().__init__(
            file_folder=file_folder, 
            file_number=file_number, 
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap, 
            search_k=search_k,
            sample_name_searcher_model_name=sample_name_searcher_model_name,
            supervisor_model_name=supervisor_model_name,
            researcher_model_name=researcher_model_name,
            verifier_model_name=verifier_model_name,
        )
        
        ## node 생성
        self.supervisor_node = functools.partial(self.agent_node, agent=self.supervisor_agent, name="Supervisor")
        self.sample_name_searcher_node = functools.partial(self.agent_node, agent=self.sample_name_searcher_chain, name="SampleNameSearcher")
        self.researcher_node = functools.partial(self.agent_node, agent=self.researcher_agent, name="Researcher")
        self.verifier_node = functools.partial(self.agent_node, agent=self.verifier_agent, name="Verifier")

        ## set tool
        self.tools = [self.retriever_tool]
        self.tool_executor = ToolExecutor(self.tools)
        
        ## graph 구축
        workflow = StateGraph(AgentState)
        workflow.add_node("SampleNameSearcher", self.sample_name_searcher_node)
        workflow.add_node("Supervisor", self.supervisor_node)
        workflow.add_edge("SampleNameSearcher", "Supervisor")
        
        for i in range(1, 5):
            workflow.add_node(f"Researcher{i}", self.researcher_node)
            workflow.add_node(f"Verifier{i}", self.verifier_node)
            workflow.add_node(f"call_tool{i}", self.tool_node)

            workflow.add_edge("Supervisor", f"Researcher{i}")
            workflow.add_conditional_edges(
                f"Researcher{i}",
                self.router,
                {"continue": f"Verifier{i}", "call_tool": f"call_tool{i}"},
            )
            workflow.add_conditional_edges(
                f"Verifier{i}",
                self.router,
                {"continue": f"Researcher{i}", "call_tool": f"call_tool{i}", "save_answer": "Supervisor"},
            )
            workflow.add_conditional_edges(
                f"call_tool{i}",
                lambda x: x["sender"],
                {
                    f"Researcher{i}": f"Researcher{i}",
                    f"Verifier{i}": f"Verifier{i}",
                },
            )
            
        workflow.set_entry_point("SampleNameSearcher")
        workflow.add_edge("Supervisor", END)
        self.graph = workflow.compile()   
        
        if save_graph_png:        
            self.graph.get_graph().draw_mermaid_png(
                output_file_path="./graph_img/supervise_multiagent_graph.png",
                padding=25
                )
    
    
    def agent_node(self, state, agent, name):
        agent_response = agent.invoke(state)
        if isinstance(agent_response, FunctionMessage):
            pass
        else:
            agent_response = HumanMessage(**agent_response.dict(exclude={"type", "name"}), name=name)
        return {
            "messages": [agent_response],
            "sender": name,
        }


    def tool_node(self, state):
        # 그래프에서 도구를 실행하는 함수입니다.
        # 에이전트 액션을 입력받아 해당 도구를 호출하고 결과를 반환합니다.
        messages = state["messages"]
        
        # 계속 조건에 따라 마지막 메시지가 함수 호출을 포함하고 있음을 알 수 있습니다.
        first_message = messages[0]
        last_message = messages[-1]
        
        # ToolInvocation을 함수 호출로부터 구성합니다.
        tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])
        tool_name = last_message.additional_kwargs["function_call"]["name"]
        
        if tool_name == "retriever":
            base_query = tool_input.get("__arg1", "")  # 기존 query 가져오기
            refined_query = f"Context: {first_message.content} | Query: {base_query}"
            tool_input["__arg1"] = refined_query
        
        # 단일 인자 입력은 값으로 직접 전달할 수 있습니다.
        if len(tool_input) == 1 and "__arg1" in tool_input:
            tool_input = next(iter(tool_input.values()))
        
        action = ToolInvocation(
            tool=tool_name,
            tool_input=tool_input,
        )
        
        # 도구 실행자를 호출하고 응답을 받습니다.
        response = self.tool_executor.invoke(action)
        
        # 응답을 사용하여 FunctionMessage를 생성합니다.
        function_message = FunctionMessage(
            content=f"{tool_name} response: {str(response)}", name=action.tool
        )
        
        # 기존 리스트에 추가될 리스트를 반환합니다.
        return {"messages": [function_message]}
    
    
    def router(self, state):
        # 상태 정보를 기반으로 다음 단계를 결정하는 라우터 함수
        messages = state["messages"]
        last_message = messages[-1]
        if "function_call" in last_message.additional_kwargs:
            return "call_tool"
        
        if "### Complete Verification" in last_message.content:
            return "save_answer"
        
        if "### Final Answer" in last_message.content:
            return "output"
        
        return "continue"

In [4]:
sample_name_question = """
Use all of the NCM cathode sample names (e.g., 'NCM-622', 'pristine NCM', 'M-NCM') provided in the electrochemical performance section. You just output sample names. Do Not output like '- NCM622' , just output 'NCM622.
"""

In [5]:
supervisor_question = """
논문에서 사용된 양극 활물질(CAM, Cathode Active Material)의 화학 조성(Stoichiometry) 정보는 무엇인가요?  
- 상업적으로 사용된 NCM(Nickel Cobalt Manganese)은 무엇인가요?  
- 사용된 리튬 원료(Lithium source)는 무엇인가요?  
- 사용된 합성 방법(Synthesis method)은 무엇인가요?  
- 결정화 방법(Crystallization method)은 무엇인가요?  
- 결정화 공정의 최종 온도(Crystallization final temperature)와 지속 시간(duration)은 각각 얼마인가요?  
- 사용된 도핑(Doping) 기술이 있는 경우, 어떤 원소가 도핑되었나요?  
- 양극 활물질에 적용된 코팅(Coating) 기술이 있다면, 어떤 물질로 코팅되었나요?  

논문에서 보고된 전극(Electrode, half-cell) 조성은 어떻게 되나요?  
- 활성 물질(Active material)과 도전제(Conductive additive), 바인더(Binder)의 비율은 얼마인가요?  
- 사용된 전해질(Electrolyte)의 종류는 무엇인가요?  
- 사용된 첨가제(Additive)는 무엇인가요?  
- NCM의 질량 적재량(Loading density, mass loading of NCM)은 얼마인가요?  

논문에서 보고된 형태학적 특성(Morphological Properties)은 무엇인가요?  
- 입자 크기(Particle size) 정보는 어떻게 되나요?  
- 입자 형태(Particle shape)는 어떻게 기술되었나요?  
- 입자 분포(Particle distribution) 특성은 어떻게 설명되었나요?  
- 코팅층(Coating layer)의 특성과 두께는 어떻게 보고되었나요?  
- 결정 구조(Crystal structure)와 격자 특성(lattice characteristics)은 무엇인가요?  

논문에서 보고된 양극 성능(Cathode Performance)은 어떻게 되나요?  
- 실험에 사용된 전압 범위(Voltage range)와 온도(Temperature)는 얼마인가요?  
- 서로 다른 C-rate에서의 용량(Specific capacity)은 어떻게 보고되었나요?  
  - 0.1C에서의 용량은?  
  - 0.2C에서의 용량은?  
  - 0.5C에서의 용량은?  
  - 1.0C에서의 용량은?  
  - 2.0C에서의 용량은?  
  - 그 외 추가적인 C-rate와 성능 데이터가 있다면 무엇인가요?  
"""

In [6]:
from langchain_core.messages import HumanMessage

invoke_input = {
    "sample_name_question" : [HumanMessage(content=sample_name_question)],
    "messages": [HumanMessage(content=supervisor_question)]
}

print(invoke_input)
print(type(invoke_input["messages"]))

{'sample_name_question': [HumanMessage(content="\nUse all of the NCM cathode sample names (e.g., 'NCM-622', 'pristine NCM', 'M-NCM') provided in the electrochemical performance section. You just output sample names. Do Not output like '- NCM622' , just output 'NCM622.\n", additional_kwargs={}, response_metadata={})], 'messages': [HumanMessage(content='\n논문에서 사용된 양극 활물질(CAM, Cathode Active Material)의 화학 조성(Stoichiometry) 정보는 무엇인가요?  \n- 상업적으로 사용된 NCM(Nickel Cobalt Manganese)은 무엇인가요?  \n- 사용된 리튬 원료(Lithium source)는 무엇인가요?  \n- 사용된 합성 방법(Synthesis method)은 무엇인가요?  \n- 결정화 방법(Crystallization method)은 무엇인가요?  \n- 결정화 공정의 최종 온도(Crystallization final temperature)와 지속 시간(duration)은 각각 얼마인가요?  \n- 사용된 도핑(Doping) 기술이 있는 경우, 어떤 원소가 도핑되었나요?  \n- 양극 활물질에 적용된 코팅(Coating) 기술이 있다면, 어떤 물질로 코팅되었나요?  \n\n논문에서 보고된 전극(Electrode, half-cell) 조성은 어떻게 되나요?  \n- 활성 물질(Active material)과 도전제(Conductive additive), 바인더(Binder)의 비율은 얼마인가요?  \n- 사용된 전해질(Electrolyte)의 종류는 무엇인가요?  \n- 사용된 첨가제(Additive)는 무엇인가요?  \n- NCM

In [7]:
voltai_graph = MultiAgentSupervisor(save_graph_png=True).graph
result = voltai_graph.invoke(invoke_input) 

TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'

In [17]:
invoke_input = (
    {
        "messages": [
            HumanMessage(content=f"{sample_name_searcher_question}", name="SampleNameSearcher"),
            HumanMessage(content=f"{supervisor_question}", name="Supervisor"),
        ],
    },
    {"recursion_limit": 30}
)

In [18]:
voltai_graph = SuperviseMultiAgentRAG().graph

In [23]:
result = voltai_graph.invoke(**invoke_input)

TypeError: langgraph.pregel.Pregel.invoke() argument after ** must be a mapping, not tuple

In [ ]:
result

In [ ]:
def main(
    data_folder:str="./data",
    file_num_list:list=[11],
    chunk_size:int=500, 
    chunk_overlap:int=100, 
    search_k:int=10,       
    config_folder:str="./config",
    rag_method:str="multiagent-rag", 
    model_name:str="gpt-4o", 
    save_graph_png:bool=False, 
):
    category_names = ["CAM (Cathode Active Material)", "Electrode (half-cell)", "Morphological Properties", "Cathode Performance"]
    
    total_outputs = {}    
    
    ## 각 논문에 대해 반복
    for file_number in file_num_list:
        total_outputs[f"paper{file_number}"] = {}
        print(f"#####    {file_number}번째 논문    #####")
        print(f"##       rag method     : {rag_method}")
        
        
        for category_number in range(1,5):
            print(f"##       Category Name   : {category_names[category_number-1]}")

            ## config 파일과 system_prompt 와 invoke_input 불러오기 (config 폴더 명 수정 필요요)
            system_prompt = load_system_prompt(config_folder=config_folder, category_number=category_number, rag_method=rag_method)
            invoke_input = load_invoke_input(config_folder=config_folder, category_number=category_number, rag_method=rag_method, sample_names=sample_names)

            ## graph 호출
            voltai_graph = get_rag_instance(
                rag_method=rag_method, 
                file_folder=f"{data_folder}/raw/", 
                file_number=file_number, 
                chunk_size=chunk_size, 
                chunk_overlap=chunk_overlap, 
                search_k=search_k, 
                system_prompt=system_prompt,
                model_name=model_name, 
                save_graph_png=save_graph_png,
            ).graph
            
            ## 질문이 딕셔너리 형태일 경우와 아닌 경우를 처리
            if isinstance(invoke_input, dict):
                result = voltai_graph.invoke(**invoke_input)
            else:
                result = voltai_graph.invoke(*invoke_input)

            ## RAG method에 따른 결과 확인
            if result.get("answer"):
                temp_answer = result["answer"][0][category_names[category_number-1]]
            elif result.get("discussion"):
                temp_answer = result["discussion"][0][category_names[category_number-1]]
            elif result.get("messages"):
                temp_answer = result["messages"][-1][category_names[category_number-1]]
            
            pprint(temp_answer, sort_dicts=False)
            
            ## json 저장하는 코드
            save_output2json(each_answer=temp_answer,file_num=file_number, rag_method=rag_method, category_number=category_number)
            
            total_outputs[f"paper{file_number}"][category_names[category_number-1]] = temp_answer
                    
    return total_outputs

In [9]:
file_num_list = [39]

In [5]:
multiagent_rag_output = main(file_num_list=file_num_list, rag_method="multiagent-rag")

#####    39번째 논문    #####
##       rag method     : multiagent-rag
##       Sample Names    : ['N92  ', 'WN92']
##          Category Name   : CAM (Cathode Active Material)
##          ./config/multiagent-rag/c1-question.yaml를 불러왔습니다.
##       print 39 result
------------------------------------
{'Stoichiometry information': {'N92': {'Li ratio': 1.0,
                                       'Ni ratio': 0.92,
                                       'Co ratio': 0.04,
                                       'Mn ratio': 0.04,
                                       'O ratio': 2.0},
                               'WN92': {'Li ratio': 1.0,
                                        'Ni ratio': 0.92,
                                        'Co ratio': 0.04,
                                        'Mn ratio': 0.04,
                                        'W ratio': 0.01,
                                        'O ratio': 2.0}},
 'Commercial NCM used': {'N92': 'no', 'WN92': 'no'},
 'Lithium source': 'Li

In [6]:
relevance_rag_output = main(file_num_list=file_num_list, rag_method="relevance-rag")

#####    39번째 논문    #####
##       rag method     : relevance-rag
##       Sample Names    : ['N92', 'WN92']
##          Category Name   : CAM (Cathode Active Material)
##          ./config/relevance-rag/c1-question.yaml를 불러왔습니다.
        RELEVANCE CHECK : yes
##       print 39 result
------------------------------------
{'Stoichiometry information': {'N92': {'Li ratio': 1.0,
                                       'Ni ratio': 0.92,
                                       'Co ratio': 0.04,
                                       'Mn ratio': 0.04,
                                       'W ratio': 0.0,
                                       'O ratio': 2.0},
                               'WN92': {'Li ratio': 1.0,
                                        'Ni ratio': 0.92,
                                        'Co ratio': 0.04,
                                        'Mn ratio': 0.04,
                                        'W ratio': 0.01,
                                        'O ratio': 2

In [11]:
ensemble_rag_output = main(file_num_list=file_num_list, rag_method="ensemble-rag")

#####    39번째 논문    #####
##       rag method     : ensemble-rag
##       Sample Names    : ['N92  ', 'WN92']
##       Category Name   : CAM (Cathode Active Material)
##          ./config/ensemble-rag/c1-question.yaml를 불러왔습니다.
        RELEVANCE CHECK for ANSWER 3 : yes
        RELEVANCE CHECK for ANSWER 1 : yes
        RELEVANCE CHECK for ANSWER 2 : yes
{'Stoichiometry information': {'N92': {'Li ratio': 1.0,
                                       'Ni ratio': 0.92,
                                       'Co ratio': 0.04,
                                       'Mn ratio': 0.04,
                                       'W ratio': 0.0,
                                       'O ratio': 2.0},
                               'WN92': {'Li ratio': 1.0,
                                        'Ni ratio': 0.9198,
                                        'Co ratio': 0.04,
                                        'Mn ratio': 0.04,
                                        'W ratio': 0.001,
               